In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print("Libraries imported successfully!")


Libraries imported successfully!


In [11]:
# Display the full dataframe (optional — may be large)
df = pd.read_csv("credit_risk_enchmark_dataset_cleaned.csv")

# Check dataset dimensions
print("Shape:", df.shape)

# View column names
print("Columns:", df.columns.tolist())

# Preview first 5 rows
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'credit_risk_enchmark_dataset_cleaned.csv'